# Agents for Amazon Bedrock - create agent

This notebook provides sample code for building an Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will create a restaurant assistant that allows customers to create, delete or get reservation information. The architecture looks as following:

![Agent Architecture](./images/agent-architecture_1.png)

### Notebook Walk-through

In this notebook we will:
- Choose our Agent's underline foundation model
- Create a dynamoDB table to store the reservation details
- Create a lambda function that handles the restaurant bookings
- Create an agent
- Create an action group and associate it with the agent
- Test the agent invocation


### Next Steps: 
In the next lab, we will add a Knowledge Base to our agent and work with Prompt Attributes to provide extra information to an agent invocation call


### Pre-requisites
This notebook requires permissions to:
- create and delete Amazon IAM roles
- create lambda functions
- create dynamoDB tables
- access Amazon Bedrock

If running on SageMaker Studio, you should add the following managed policies to your role:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure to enable `Anthropic Claude 3 Sonnet` model access in Amazon Bedrock Console, as the notebook will use Anthropic Claude 3 Sonnet model for testing the agent once its created.
</div>


## Setup
Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed

In [1]:
!pip install --upgrade -q -r requirements.txt

Let's now import the necessary libraries and initiate the required boto3 clients

In [1]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

In [2]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '165000421549')

### Setting up Agent's information

We will now set the variables that define our agent:

- **agent_name**: provides the name of the agent to be created, in this case `booking-agent`
- **agent_description**: the description of the agent used to display the agents list on the console. This description is **not** part of the agent's prompts
- **agent_instruction**: the instructions of what the agent should and should not do. This description is part of the agent's prompt and is used during the agent's invocation
- **agent_action_group_name**: the action group name used on the definition of the agent's action, in this case `TableBookingsActionGroup`.
- **agent_action_group_description:**: the description of the action group name used on the UI to list the action groups. This description is **not** used by the agent's prompts

In [15]:
suffix = f"{region}-{account_id}"
agent_name = 'scheduling-agent'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of managing schedule conflicts and arranging suitable times for meetings"
agent_instruction = """
You are a schedule management agent, in charge of tracking user availabilities, creating events, tracking user RSVP to events, and suggesting optimal times for events based on the availability of users who RSVP'ed.
"""

agent_action_group_description = """
Actions for setting user availability, creating events, setting user RSVP, and suggesting a meeting time"""

agent_action_group_name = "ScheduleMeetingActionGroup"

### Select Foundation Model
Use this dropdown menu to select the underline model of your agent. You can find more information about the supported foundation models [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-supported.html)

In [16]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

Dropdown(description='FM:', options=(('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'), ('Claude …

Let's confirm that the model has been selected correctly

In [17]:
agent_foundation_model = agent_foundation_model_selector.value
agent_foundation_model

'anthropic.claude-3-sonnet-20240229-v1:0'

### Creating DynamoDB table

Let's now create an [Amazon DynamoDB](https://aws.amazon.com/dynamodb/) table called `restaurant_bookings`. This table will store information about the reservations, including a `booking_id`, reservation `date`, the `name` of the person doing the reservation, the `hour` of the reservation and the number of guests as `num_guests`. To do so, we use the `create_dynamodb` function from the `agent.py` file. This function will support the creation of the table and its requirements (IAM roles and permissions).

In [8]:
# table_name = 'restaurant_bookings'
# create_dynamodb(table_name)

### Creating Lambda Function

Next we will create the [AWS Lambda](https://aws.amazon.com/lambda/) function that executes the actions for our agent. This lambda function will have 3 actions:
* ```get_booking_details(booking_id)```: returns the details of a booking based on the booking id
* ```create_booking(date, name, hour, num_guests)```: creates a new booking for the restaurant
* ```delete_booking(booking_id)```: deletes an existent booking based on the booking id


The `lambda_handler` receives the `event` from the agent and the `event` contains information about the `function` to be executed and its `parameters`. 

A `functionResponse` is returned by the lambda function with the response body having a `TEXT` field.

You can find more information on how to set your agent lambda function [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-lambda.html).

Let's first write the code of the lambda function to the `lambda_function.py` file

In [4]:
%%writefile lambda_function.py
import os
import json
from supabase import create_client, Client
from datetime import datetime, timedelta

# Initialize Supabase client
url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(url, key)

def get_named_parameter(event, name):
    """
    Get a parameter from the lambda event
    """
    return next(item for item in event['parameters'] if item['name'] == name)['value']

def create_event(group_id, period, description):
    """
    Create a new event
    
    Args:
        group_id (string): The ID of the group creating the event
        period (string): The period for the event (e.g., "2023-07-01 to 2023-07-07")
        description (string): Description of the event
    """
    try:
        data, count = supabase.table('events').insert({
            'groupID': group_id,
            'period': period,
            'description': description,
            'confirmedTime': None  # This will be updated later when RSVP is processed
        }).execute()
        
        return {'eventID': data[1][0]['eventID']}
    except Exception as e:
        return {'error': str(e)}

def parse_datetime(dt_string):
        return datetime.strptime(dt_string.lstrip("start:"), "%y-%m-%d:%H-%M").isoformat()

def rsvp(user_id, event_id, availiable_time):
    """
    Indicate a user's availability for an event
    
    Args:
        user_id (string): The ID of the user
        event_id (string): The ID of the event
        availiable_time (string): The user's available time range. The format is pairs of start:YYYY-MM-DD:HH-MM and end:YYYY-MM-DD:HH-MM indicating the start and end of the user's available time ranges. If the user has multiple available time ranges, submit multiple pairs
    """
    try:
        for i in range(len(available_time.split(), 2):
            try:
                start_time = parse_datetime(availiable_time[i])
                end_time = parse_datetime(availiable_time[i + 1])
            except Exception as e:
                return {'error': "Malformed available time data!" + str(e)}
            
        data, count = supabase.table('availabilities').insert({
            'userID': user_id,
            'eventID': event_id,
            'startTime': start_time,
            'endTime': end_time
        }).execute()
        
        return {'message': 'Availability recorded successfully'}
    except Exception as e:
        return {'error': " + str(e)}

def find_best_time(event_id):
    """
    Find the time with the greatest overlapping time ranges for an event
    
    Args:
        event_id (string): The ID of the event
    """
    try:
        # Fetch all availabilities for the event
        data, count = supabase.table('availabilities').select('*').eq('eventID', event_id).execute()
        availabilities = data[1]

        if not availabilities:
            return None

        # Convert string times to datetime objects
        for avail in availabilities:
            avail['startTime'] = datetime.fromisoformat(avail['startTime'])
            avail['endTime'] = datetime.fromisoformat(avail['endTime'])

        # Find the overall start and end times
        start_time = min(avail['startTime'] for avail in availabilities)
        end_time = max(avail['endTime'] for avail in availabilities)

        # Create a list of all possible 30-minute slots
        slots = []
        current_time = start_time
        while current_time < end_time:
            slots.append((current_time, current_time + timedelta(minutes=30)))
            current_time += timedelta(minutes=30)

        # Count the number of people available for each slot
        best_slot = max(slots, key=lambda slot: sum(
            1 for avail in availabilities if avail['startTime'] <= slot[0] and avail['endTime'] >= slot[1]
        ))

        return best_slot[0].isoformat()

    except Exception as e:
        print(f"Error finding best time: {str(e)}")
        return None

def update_event_time(event_id):
    """
    Update the confirmed time for an event
    
    Args:
        event_id (string): The ID of the event
        confirmed_time (string): The confirmed time for the event
    """
    try:
        data, count = supabase.table('events').update({
            'confirmedTime': confirmed_time
        }).eq('eventID', event_id).execute()
        
        return {'message': 'Event time updated successfully'}
    except Exception as e:
        return {'error': str(e)}

def select_time(event_id):
    """
    Process RSVP for an event
    
    Args:
        event_id (string): The ID of the event
    """
    best_time = find_best_time(event_id)
    if best_time:
        return update_event_time(event_id, best_time)
    else:
        return {'error': 'Unable to find a suitable time'}

def lambda_handler(event, context):
    # Get the action group used during the invocation of the lambda function
    action_group = event.get('actionGroup', '')
    
    # Name of the function that should be invoked
    function = event.get('function', '')
    
    if function == 'create_event':
        group_id = get_named_parameter(event, "group_id")
        period = get_named_parameter(event, "period")
        description = get_named_parameter(event, "description")

        if group_id and period and description:
            response = create_event(group_id, period, description)
            response_body = {'TEXT': {'body': json.dumps(response)}}
        else:
            response_body = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'rsvp':
        user_id = get_named_parameter(event, "user_id")
        event_id = get_named_parameter(event, "event_id")
        availiable_time = get_named_parameter(event, "availiable_time")

        if user_id and event_id and start_time and end_time:
            response = rsvp(user_id, event_id, availiable_time)
            response_body = {'TEXT': {'body': json.dumps(response)}}
        else:
            response_body = {'TEXT': {'body': 'Missing required parameters'}}

    elif function == 'find_best_time`':
        event_id = get_named_parameter(event, "event_id")

        if event_id:
            response = find_best_time(event_id)
            response_body = {'TEXT': {'body': json.dumps(response)}}
        else:
            response_body = {'TEXT': {'body': 'Missing event_id parameter'}}

    else:
        response_body = {'TEXT': {'body': 'Invalid function'}}

    action_response = {
        'actionGroup': action_group,
        'function': function,
        'functionResponse': {
            'responseBody': response_body
        }
    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response

Overwriting lambda_function.py


In [5]:
# %%writefile lambda_function.py
# import json
# import uuid
# import boto3

# dynamodb = boto3.resource('dynamodb')
# table = dynamodb.Table('restaurant_bookings')

# def get_named_parameter(event, name):
#     """
#     Get a parameter from the lambda event
#     """
#     return next(item for item in event['parameters'] if item['name'] == name)['value']


# def get_booking_details(booking_id):
#     """
#     Retrieve details of a restaurant booking
    
#     Args:
#         booking_id (string): The ID of the booking to retrieve
#     """
#     try:
#         response = table.get_item(Key={'booking_id': booking_id})
#         if 'Item' in response:
#             return response['Item']
#         else:
#             return {'message': f'No booking found with ID {booking_id}'}
#     except Exception as e:
#         return {'error': str(e)}


# def create_booking(date, name, hour, num_guests):
#     """
#     Create a new restaurant booking
    
#     Args:
#         date (string): The date of the booking
#         name (string): Name to idenfity your reservation
#         hour (string): The hour of the booking
#         num_guests (integer): The number of guests for the booking
#     """
#     try:
#         booking_id = str(uuid.uuid4())[:8]
#         table.put_item(
#             Item={
#                 'booking_id': booking_id,
#                 'date': date,
#                 'name': name,
#                 'hour': hour,
#                 'num_guests': num_guests
#             }
#         )
#         return {'booking_id': booking_id}
#     except Exception as e:
#         return {'error': str(e)}


# def delete_booking(booking_id):
#     """
#     Delete an existing restaurant booking
    
#     Args:
#         booking_id (str): The ID of the booking to delete
#     """
#     try:
#         response = table.delete_item(Key={'booking_id': booking_id})
#         if response['ResponseMetadata']['HTTPStatusCode'] == 200:
#             return {'message': f'Booking with ID {booking_id} deleted successfully'}
#         else:
#             return {'message': f'Failed to delete booking with ID {booking_id}'}
#     except Exception as e:
#         return {'error': str(e)}
    

# def lambda_handler(event, context):
#     # get the action group used during the invocation of the lambda function
#     actionGroup = event.get('actionGroup', '')
    
#     # name of the function that should be invoked
#     function = event.get('function', '')
    
#     # parameters to invoke function with
#     parameters = event.get('parameters', [])

#     if function == 'get_booking_details':
#         booking_id = get_named_parameter(event, "booking_id")
#         if booking_id:
#             response = str(get_booking_details(booking_id))
#             responseBody = {'TEXT': {'body': json.dumps(response)}}
#         else:
#             responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

#     elif function == 'create_booking':
#         date = get_named_parameter(event, "date")
#         name = get_named_parameter(event, "name")
#         hour = get_named_parameter(event, "hour")
#         num_guests = get_named_parameter(event, "num_guests")

#         if date and hour and num_guests:
#             response = str(create_booking(date, name, hour, num_guests))
#             responseBody = {'TEXT': {'body': json.dumps(response)}}
#         else:
#             responseBody = {'TEXT': {'body': 'Missing required parameters'}}

#     elif function == 'delete_booking':
#         booking_id = get_named_parameter(event, "booking_id")
#         if booking_id:
#             response = str(delete_booking(booking_id))
#             responseBody = {'TEXT': {'body': json.dumps(response)}}
#         else:
#             responseBody = {'TEXT': {'body': 'Missing booking_id parameter'}}

#     else:
#         responseBody = {'TEXT': {'body': 'Invalid function'}}

#     action_response = {
#         'actionGroup': actionGroup,
#         'function': function,
#         'functionResponse': {
#             'responseBody': responseBody
#         }
#     }

#     function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
#     print("Response: {}".format(function_response))

#     return function_response

Next we create the function requirements for IAM role and policies using the support function `create_lambda_role` and create the lambda using the support function `create_lambda` both from the `agent.py` file

In [6]:
lambda_iam_role = create_lambda_role(agent_name)

In [7]:
lambda_function_name = f'{agent_name}-lambda'

In [8]:
lambda_function = create_lambda(lambda_function_name, lambda_iam_role)

### Creating Agent

Now that we have created the dynamoDB table and lambda function, let's create our Agent. 

To do so, we first need to create an agent role and its required policies. Let's do so using the `create_agent_role` function from the `agent.py` file.

In [18]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [19]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_scheduling-agent',
  'RoleId': 'AROASM2WKUCW4LVQ6QN45',
  'Arn': 'arn:aws:iam::165000421549:role/AmazonBedrockExecutionRoleForAgents_scheduling-agent',
  'CreateDate': datetime.datetime(2024, 7, 12, 18, 55, 41, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600,
  'RoleLastUsed': {}},
 'ResponseMetadata': {'RequestId': 'd349d7a5-596e-4b05-b0a0-371d2b5af168',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 14 Jul 2024 02:49:25 GMT',
   'x-amzn-requestid': 'd349d7a5-596e-4b05-b0a0-371d2b5af168',
   'content-type': 'text/xml',
   'content-length': '894'},
  'RetryAttempts': 0}}

With the Agent IAM role created, we can now use the boto3 function [`create_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent.html) to create our agent. 

On the agent creation, all you need to provide is the agent name, foundation model and instruction. We will associate an action group to the agent once it has been created

In [22]:
# response = bedrock_agent_client.create_agent(
#     agentName=agent_name,
#     agentResourceRoleArn=agent_role['Role']['Arn'],
#     description=agent_description,
#     idleSessionTTLInSeconds=1800,
#     foundationModel=agent_foundation_model,
#     instruction=agent_instruction,
# )
# response

In [26]:
response = bedrock_agent_client.list_agents()

if response['agentSummaries']:
    agent_id = response['agentSummaries'][0]['agentId']
    print(f"Agent ID: {agent_id}")
else:
    print(f"No agent found with name starting with '{agent_name}'")

Agent ID: BMYPHRKQ4B


Now that our agent has been created, we will retrieve the `agentId`. It will be used to associate the action group to the agent in our next step.

In [28]:
# agent_id = response['agent']['agentId']
# print("The agent id is:",agent_id)

#### Create Agent Action Group

now that we have created the agent, let's create an [Action Group](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) and associate with the agent. The action group will allow our agent to execute the booking tasks. To do so, we will "inform" our agent about the existent functionalities using a [function schema](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) defined in `JSON` format.

The function schema requires the function `name`, `description` and `parameters` to be provided. Each parameter has a parameter name, description, type and a boolean flag indicating if the parameter is required.

Let's define the functions `JSON` as `agent_functions`

In [54]:
agent_functions = [
    {
        'name': 'create_event',
        'description': 'Create an event for users to attend',
        'parameters': {
            "period": {
                "description": "duration of the event in number of hours",
                "required": True,
                "type": "string"
            },
            "description": {
                "description": "A description of what the event activities will be",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'rsvp',
        'description': 'Accept user RSVP for a event',
        'parameters': {
            "user_id": {
                "description": "The ID of the user sending the RSVP",
                "required": True,
                "type": "string"
            },
            "event_id": {
                "description": "The ID of the event the user is RSVPing for",
                "required": True,
                "type": "string"
            },
            "availiable_time": {
                "description": "The user's available time range. The format is pairs of start:YYYY-MM-DD::HH-MM and end:YYYY-MM-DD::HH-MM indicating the start and end of the user's available time ranges. If the user has multiple available time ranges, submit multiple pairs",
                "required": True,
                "type": "string"
            },
        }
    },
    {
        'name': 'find_best_time',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
             "event_id": {
                "description": "The ID of the event to find the best time for",
                "required": True,
                "type": "string"
            }
        }
    },
]

Now we can use the [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function from the boto3 SDK to create the action group

In [55]:
list_response = bedrock_agent_client.list_agent_action_groups(
    agentId=agent_id,
    agentVersion='DRAFT'
)

# Check if an action group with the given name exists
action_group_id = next((group['actionGroupId'] for group in list_response['actionGroupSummaries'] 
                              if group['actionGroupName'] == agent_action_group_name), None)

action_group_id

'6FIQR98BRU'

In [56]:
# Pause to make sure agent is created
# time.sleep(30)

# Now, we can configure and create an action group here:
if action_group_id:
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function['FunctionArn']
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
else:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
        agentId=agent_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function['FunctionArn']
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )

In [57]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '57a98b59-9b0a-490e-b2fe-c7c770ae74e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 14 Jul 2024 03:20:13 GMT',
   'content-type': 'application/json',
   'content-length': '1673',
   'connection': 'keep-alive',
   'x-amzn-requestid': '57a98b59-9b0a-490e-b2fe-c7c770ae74e3',
   'x-amz-apigw-id': 'a4eNKEOMvHcEGKg=',
   'x-amzn-trace-id': 'Root=1-669343ed-0b73c7cb6a76b3b92ba41179'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:165000421549:function:scheduling-agent-lambda'},
  'actionGroupId': '6FIQR98BRU',
  'actionGroupName': 'ScheduleMeetingActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'BMYPHRKQ4B',
  'agentVersion': 'DRAFT',
  'clientToken': '437d3917-ea3b-4459-b2d4-91ff8eb9679d',
  'createdAt': datetime.datetime(2024, 7, 14, 2, 56, 4, 397263, tzinfo=tzutc()),
  'description': '\nActions for setting user availability, creating events, setting user RSVP, and suggestin

#### Allowing bedrock to invoke lambda function

The last requirement is to add the [resource-based policy](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-permissions.html#agents-permissions-lambda) to allow bedrock to invoke the action group lambda function.

In [58]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

An error occurred (ResourceConflictException) when calling the AddPermission operation: The statement id (allow_bedrock_BMYPHRKQ4B) provided already exists. Please provide a new statement id, or remove the existing statement.


#### Preparing agent

Before invoking the agent we need to prepare it. Preparing your agent will package all its components, including the security configurations. It will bring the agent into a state where it can be tested in runtime. We will use the [`prepare_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/prepare_agent.html) function from the boto3 sdk to prepare our agent.

In [49]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': 'a8753404-cd17-4d69-ac71-f5b2979657e1', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 14 Jul 2024 03:04:13 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'a8753404-cd17-4d69-ac71-f5b2979657e1', 'x-amz-apigw-id': 'a4b3KHPyPHcEmkw=', 'x-amzn-trace-id': 'Root=1-6693402d-0a03a14f4b94cca3383b8819'}, 'RetryAttempts': 0}, 'agentId': 'BMYPHRKQ4B', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 7, 14, 3, 4, 13, 790784, tzinfo=tzutc())}


### Invoking Agent

Now that our Agent is ready to be used, let's test it. To do so we will use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

To invoke an agent, you have to refer to its alias. You can create a new alias, or you can use the test alias to invoke your `DRAFT` agent. The test alias used to invoke the draft agent is `TSTALIASID` and it will work with any agent. 


We will use the support function called `invoke_agent_helper` from the `agents.py` support file to allow us to invoke the agent with or without trace enabled and with or without session state. We will getinto more details about those concepts in the `03_invoke_agent.ipynb` notebook.

In [50]:
alias_id = 'TSTALIASID'

In [63]:
%%time
session_id:str = str(uuid.uuid1())
query = "<userID>21381832</userID> Hi, I am Anna. I want to create an event anytime in the next week, for an hour long"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Exception: ('unexpected event.', EventStreamError("An error occurred (dependencyFailedException) when calling the InvokeAgent operation: Your request couldn't be completed. Lambda function arn:aws:lambda:us-west-2:165000421549:function:scheduling-agent-lambda encountered a problem while processing request.The error message from the Lambda function is Unhandled. Check the Lambda function log for error details, then try your request again after fixing the error."))

### Next Steps

Before moving to the next notebook, let's store a couple of variables to continue working the the same notebook.

Next we will update our agent to associate a knowledge base containing the menus for our restaurant. We will then test the agent invocations and clean up all the created resources.

In [ ]:
%store agent_id
%store agent_role
%store lambda_iam_role
%store agent_name
%store suffix
%store region
%store agent_foundation_model
%store account_id
%store alias_id
%store table_name
%store lambda_function
%store lambda_function_name
%store agent_action_group_response
%store agent_functions